# Start

In [1]:
import pandas as pd
import numpy as np
import missingno as msno
from ydata_profiling import ProfileReport
import plotly_express as px
from itertools import combinations
from collections import Counter
from sklearn.cluster import KMeans


## Datasets

In [2]:
zip_path1 = "public/aisles.csv.zip"
zip_path2 = "public/departments.csv.zip"
zip_path3 = "public/order_products_s.csv.zip"
zip_path4 = "public/orders_s.csv.zip"
zip_path5 = "public/products.csv.zip"
zip_path6 = "public/tips.csv.zip"

df_aisles = pd.read_csv(zip_path1, compression="zip")
df_departments = pd.read_csv(zip_path2, compression="zip")
df_order_products = pd.read_csv(zip_path3, compression="zip")
df_oders_s = pd.read_csv(zip_path4, compression="zip")
df_products = pd.read_csv(zip_path5, compression="zip")
df_tips = pd.read_csv("public/tips.csv")

In [3]:
product_details_df = pd.read_csv("product_details.csv")
order_product_details_df = pd.read_csv("order_product_details.csv")
order_product_df = pd.read_csv("full_order_product.csv")
full_df = pd.read_csv("full_data.csv")

# Analyse des Trinkgeld-Verhaltens

## Bestellungen mit und ohne Trinkgeld nach Wochentag und Uhrzeit
Mithilfe einer Heatmap ist man in der Lage, Spitzenzeiten zu erkennen, in denen Trinkgeld gegeben oder nicht gegeben wird und identifizieren potenziell wichtige Zeiträume

In [5]:
# Wochentag und Stunde des Trinkgelds
tip_by_dow_hour = full_df.groupby(['order_dow', 'order_hour_of_day', 'tip']).size().reset_index(name='count')
tip_by_dow_hour['tip'] = tip_by_dow_hour['tip'].map({True: "Mit Trinkgeld", False: "Ohne Trinkgeld"})

# Visualisierung: Bestellungen mit und ohne Trinkgeld nach Wochentag und Uhrzeit
fig = px.density_heatmap(
    tip_by_dow_hour, 
    x='order_hour_of_day', 
    y='order_dow', 
    z='count', 
    facet_col='tip', 
    color_continuous_scale="Viridis", 
    title="Bestellungen mit/ohne Trinkgeld nach Stunde und Wochentag"
)
fig.show()

## Trinkgeld-Verhalten nach Produktkategorie
Ein Balkendiagramm zeigt, ob bestimmte Produktkategorien eher zu Trinkgeldzahlungen führen, was Rückschlüsse auf den Wert der gekauften Waren zulässt

In [8]:
# Trinkgeld-Verhalten nach Produktkategorie
tip_by_department = full_df.groupby(['department', 'tip']).size().reset_index(name='count')
tip_by_department['tip'] = tip_by_department['tip'].map({True: "Mit Trinkgeld", False: "Ohne Trinkgeld"})

# Visualisierung: Verteilung von Bestellungen mit und ohne Trinkgeld nach Produktkategorie
fig = px.bar(
    tip_by_department, 
    x='department', 
    y='count', 
    color='tip', 
    title="Verteilung von Bestellungen mit/ohne Trinkgeld nach Produktkategorie",
    labels={'count': 'Anzahl der Bestellungen', 'department': 'Produktkategorie'},
    barmode='group'
)
fig.show()

## Durchschnittliche Trinkgeldrate nach Stunde und Wochentag
Ein Liniendiagramm, wann die Kunden am ehesten Trinkgeld geben

In [9]:
# Statistische Untersuchung: Differenzen bei Trinkgeld nach Uhrzeit und Wochentag
# Durchschnittliche Trinkgeldrate pro Stunde und Wochentag
tip_rate_by_dow_hour = full_df.groupby(['order_dow', 'order_hour_of_day'])['tip'].mean().reset_index()
tip_rate_by_dow_hour.columns = ['Wochentag', 'Stunde', 'Trinkgeldrate']

# Visualisierung: Durchschnittliche Trinkgeldrate nach Stunde und Wochentag
fig = px.line(
    tip_rate_by_dow_hour, 
    x='Stunde', 
    y='Trinkgeldrate', 
    color='Wochentag', 
    title="Durchschnittliche Trinkgeldrate nach Stunde und Wochentag"
)
fig.show()

## Tendez der Kunden, Trinkgeld zu geben, nach Warenkorbgröße
Eine Histogramm-Analyse untersucht, ob größere Bestellungen eher Trinkgeldzahlungen begünstigen.

In [ ]:
# Anzahl der Produkte in Bestellungen
full_df['order_size'] = full_df.groupby('order_id')['product_id'].transform('count')
tip_by_order_size = full_df.groupby(['order_size', 'tip']).size().reset_index(name='count')
tip_by_order_size['tip'] = tip_by_order_size['tip'].map({True: "Mit Trinkgeld", False: "Ohne Trinkgeld"})

# Visualisierung: Trinkgeldrate nach Anzahl der Produkte im Warenkorb
fig = px.histogram(
    tip_by_order_size, 
    x='order_size', 
    y='count', 
    color='tip', 
    title="Trinkgeldrate nach Anzahl der Produkte im Warenkorb",
    labels={'order_size': 'Warenkorbgröße', 'count': 'Anzahl der Bestellungen'},
    barmode='overlay'
)
fig.show()